In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# clone and install Mask2Former
!git clone https://github.com/facebookresearch/Mask2Former.git
%cd Mask2Former
!pip install -U opencv-python
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install -r requirements.txt
%cd mask2former/modeling/pixel_decoder/ops
!python setup.py build install
%cd ../../../../

fatal: destination path 'Mask2Former' already exists and is not an empty directory.
/content/Mask2Former
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-iamaw80i
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-iamaw80i
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0
  Preparing metadata (setup.py) ... done
/content/Mask2Former/mask2former/modeling/pixel_decoder/ops
running build
running build_py
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-310/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-310/functions
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-310/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-cpython-310/modules
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted 

In [3]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-w74wef37
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-w74wef37
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5b72c27ae39f99db75d43f18fd1312e1ea934e60
  Preparing metadata (setup.py) ... done


In [4]:
%cd mask2former/modeling/pixel_decoder/ops

/content/Mask2Former/mask2former/modeling/pixel_decoder/ops


In [5]:
!sh make.sh

running build
running build_py
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-310/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-310/functions
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-310/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-cpython-310/modules
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:414: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10

In [6]:
%cd ../../../..

/content/Mask2Former


In [7]:
# import Mask2Former project
from mask2former import add_maskformer2_config

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="mask2former")

# import some common libraries
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.projects.deeplab import add_deeplab_config

In [11]:
data_dir_path = "/content/drive/MyDrive/instseg/data/"

In [12]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("taco10_train", {}, data_dir_path + "mapped_annotations_0_train.json", data_dir_path + "images/")
register_coco_instances("taco10_val", {}, data_dir_path + "mapped_annotations_0_val.json", data_dir_path + "images/")
register_coco_instances("taco10_test", {}, data_dir_path + "mapped_annotations_0_test.json", data_dir_path + "images/")

# Fine-tune

In [13]:
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data import detection_utils as utils
from detectron2.structures import PolygonMasks
import copy
import torch  # Import torch to convert images to tensors
from argparse import ArgumentParser
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultTrainer
from detectron2.projects.deeplab import add_deeplab_config
import detectron2.utils.comm as comm
from detectron2.utils.logger import setup_logger
from mask2former import (
    MaskFormerInstanceDatasetMapper,
    InstanceSegEvaluator,
    add_maskformer2_config,
)
from detectron2.evaluation import COCOEvaluator

In [14]:
class Trainer(DefaultTrainer):
    """
    Extension of the Trainer class adapted to MaskFormer.
    """
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = MaskFormerInstanceDatasetMapper(cfg, True)
        return build_detection_train_loader(cfg, mapper=mapper)


    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
      if output_folder is None:
              output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
      print(dataset_name)
      return COCOEvaluator(dataset_name, output_dir=output_folder)

In [15]:
import os
from detectron2.config import get_cfg

cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cfg.merge_from_file("configs/coco/instance-segmentation/maskformer2_R50_bs16_50ep.yaml")

In [16]:
cfg.DATASETS.TRAIN = ("taco10_train",)
cfg.DATASETS.TEST = ("taco10_val",)

cfg.DATALOADER.NUM_WORKERS = 2

cfg.INPUT.DATASET_MAPPER_NAME = "mask_former_instance"

cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 10
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/instance/maskformer2_R50_bs16_50ep/model_final_3c8ec9.pkl"
cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE = "value"
cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 1.0
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 12000
cfg.SOLVER.STEPS = []
cfg.OUTPUT_DIR = "./output"
cfg.TEST.EVAL_PERIOD = 1200
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)

[09/03 08:51:20 d2.engine.defaults]: Model:
MaskFormer(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      

model_final_3c8ec9.pkl: 176MB [00:00, 204MB/s]                           


WARNING [09/03 08:51:23 mask2former.modeling.transformer_decoder.mask2former_transformer_decoder]: Weight format of MultiScaleMaskedTransformerDecoder have changed! Please upgrade your models. Applying automatic conversion now ...


criterion.empty_weight
sem_seg_head.predictor.class_embed.{bias, weight}


In [17]:
trainer.train()

Streaming output truncated to the last 5000 lines.
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv2): Conv2d(
          256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv3): Conv2d(
          256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
        )
      )
      (5): BottleneckBlock(
        (conv1): Conv2d(
          1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv2): Conv2d(
          256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv3): Conv2d(
          256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_feature

In [21]:
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("taco10_test", output_dir="output")
mapper = MaskFormerInstanceDatasetMapper(cfg, True)
test_loader = build_detection_test_loader(cfg, "taco10_test", mapper=mapper)
metrics = inference_on_dataset(trainer.model, test_loader, evaluator)
print(metrics)

[09/03 12:25:14 mask2former.data.dataset_mappers.mask_former_instance_dataset_mapper]: [MaskFormerInstanceDatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(800,), max_size=1333, sample_style='choice'), RandomFlip()]
[09/03 12:25:14 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/drive/MyDrive/instseg/data/mapped_annotations_0_test.json
[09/03 12:25:14 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/03 12:25:14 d2.data.common]: Serializing 150 elements to byte tensors and concatenating them all ...
[09/03 12:25:14 d2.data.common]: Serialized dataset takes 0.25 MiB
[09/03 12:25:14 d2.evaluation.evaluator]: Start inference on 150 batches
[09/03 12:25:53 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0027 s/iter. Inference: 0.1040 s/iter. Eval: 3.7769 s/iter. Total: 3.8836 s/iter. ETA=0:08:59
[09/03 12:26:00 d2.evaluation.evaluator]: Inference done 13/150. D